In [36]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pandas_datareader as pdr
import matplotlib as plt
import openpyxl as opl
import statistics as st
import scipy as sp
from itertools import product
from scipy import stats
import finance_byu as fin
import scipy.stats
from scipy.stats import f

In [37]:
portfolio = pd.read_excel(r'C:\Users\aless\OneDrive\Desktop\Oxford\Courses\Asset Pricing\Assignment I\25_Portfolios_5x5.xlsx', nrows= 1146, skiprows = 15, index_col = 0, parse_dates= True)
Mom = pd.read_excel(r'C:\Users\aless\OneDrive\Desktop\Oxford\Courses\Asset Pricing\Assignment I\P2\F-F_Momentum_Factor.xlsx', skiprows = 13, index_col=0, usecols='A:B')
Five_Factors = pd.read_excel(r'C:\Users\aless\OneDrive\Desktop\Oxford\Courses\Asset Pricing\Assignment I\P2\F-F_Research_Data_5_Factors_2x3.xlsx', skiprows = 3, index_col= 0)
Three_Factors = pd.read_excel(r'C:\Users\aless\OneDrive\Desktop\Oxford\Courses\Asset Pricing\Assignment I\P2\F-F_Research_Data_Factors.xlsx', skiprows= 3, index_col=0)
ST_Reversal = pd.read_excel(r'C:\Users\aless\OneDrive\Desktop\Oxford\Courses\Asset Pricing\Assignment I\P2\F-F_ST_Reversal_Factor.xlsx', skiprows = 13, index_col=0, usecols='A:B')

CHANGE DATES INTO YYYY/MM/DD FORMAT

In [38]:
portfolio.index.astype(int)
portfolio.index // 100
portfolio.index % 100

dates = []
for value in portfolio.index:
    year = value // 100
    month = value % 100
    dates.append(pd.Timestamp(year=year, month=month, day=1))
portfolio.index = dates

In [39]:
Mom.index.astype(int)
Mom.index // 100
Mom.index %100

dates = []
for value in Mom.index:
    year = value // 100
    month = value % 100
    dates.append(pd.Timestamp(year=year, month=month, day=1))
Mom.index = dates

In [40]:
Five_Factors.index.astype(int)
Five_Factors.index // 100
Five_Factors.index %100

dates = []
for value in Five_Factors.index:
    year = value // 100
    month = value % 100
    dates.append(pd.Timestamp(year=year, month=month, day=1))
Five_Factors.index = dates

In [41]:
Three_Factors.index.astype(int)
Three_Factors.index // 100
Three_Factors.index %100

dates = []
for value in Three_Factors.index:
    year = value // 100
    month = value % 100
    dates.append(pd.Timestamp(year=year, month=month, day=1))
Three_Factors.index = dates

In [42]:
ST_Reversal.index.astype(int)
ST_Reversal.index // 100
ST_Reversal.index %100

dates = []
for value in ST_Reversal.index:
    year = value // 100
    month = value % 100
    dates.append(pd.Timestamp(year=year, month=month, day=1))
ST_Reversal.index = dates

CREATE DATAFRAME FOR REGRESSIONS - 5 FACTORS MODEL [ Rit - RF = a + beta1(Rm - RF) + beta2(SMBt) + beta3(HMLt) + beta4(RMWt) + beta5(CMAt) ]

In [43]:
risk_free = Five_Factors['RF']
Five_Factors_w_constant = sm.add_constant(Five_Factors)
Five_Factors_w_constant = Five_Factors_w_constant.iloc[:,:-1]

In [44]:
port_excess_r = []
for column in portfolio:
    portfolio_new = portfolio[column] - risk_free
    port_excess_r.append(portfolio_new)

port_excess_r = pd.DataFrame(port_excess_r)
port_excess_r = port_excess_r.T
col_names = [(portfolio.columns)]
port_excess_r.columns = ['SMALL LoBM', 'ME1 BM2', 'ME1 BM3', 'ME1 BM4', 'SMALL HiBM', 'ME2 BM1', 'ME2 BM2', 'ME2 BM3', 'ME2 BM4', 'ME2 BM5', 'ME3 BM1', 'ME3 BM2', 'ME3 BM3', 'ME3 BM4', 'ME3 BM5', 'ME4 BM1', 'ME4 BM2', 'ME4 BM3', 'ME4 BM4', 'ME4 BM5', 'BIG LoBM', 'ME5 BM2', 'ME5 BM3', 'ME5 BM4','BIG HiBM']
port_excess_r


,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
1963-07-01,1.4388,-0.4352,0.1910,-0.2734,-1.4885,-2.6285,-0.0801,-1.2237,-2.2285,-1.5168,...,-1.2506,-1.7214,-2.1884,-1.8595,-2.1267,-0.1153,0.1653,0.9857,-0.6990,-1.3726
1963-08-01,4.5361,1.2629,1.0807,2.0003,4.5524,4.9361,4.2691,4.2771,4.5149,7.6114,...,5.1759,4.4410,5.9832,7.4888,5.0969,5.5191,3.9711,4.4987,7.8919,6.1324
1963-09-01,-3.2662,0.3702,-1.1541,-1.7954,-0.7153,-4.2886,-1.7734,-1.2215,-1.6204,-3.1048,...,-3.0674,-2.4285,-2.0500,-4.3365,-2.2643,-1.6295,-1.0185,-1.2552,-0.4605,-3.7663
1963-10-01,0.7174,-0.8904,0.8338,-0.0973,2.1297,1.1228,3.9442,2.1278,1.8055,3.5733,...,-0.6802,0.3973,2.3329,4.5088,0.3213,5.0439,1.4528,-0.5729,2.1250,0.1957
1963-11-01,-3.8678,-4.3345,-1.8748,-1.2972,-1.3898,-4.3869,-2.0234,-1.0496,-0.4243,-0.3910,...,-1.1498,-0.9488,-1.0633,1.3499,3.2688,-1.5256,0.7268,-1.8242,-2.4022,1.0755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-01,9.0335,5.9056,10.4974,11.4966,-3.1854,3.9416,8.8313,9.0869,11.6624,9.3689,...,3.4750,5.3391,9.1333,12.2373,12.6722,-0.6013,3.2736,6.7689,8.3882,14.3221
2021-03-01,-5.0924,1.1965,2.0749,6.1428,11.5952,-5.2298,-0.8036,2.3836,4.7201,7.5352,...,-1.9502,4.1918,3.8556,5.3175,6.5577,1.8242,4.9932,5.9902,7.3866,6.1788
2021-04-01,-3.1918,-2.5146,1.6436,1.0073,-0.9711,3.5662,1.6463,2.2390,0.7952,1.2227,...,4.3807,3.1023,5.9322,3.3530,6.0459,5.8966,5.8241,3.9870,4.7821,4.5701
2021-05-01,1.4358,0.5285,0.7486,2.3463,10.4483,-2.4340,0.1611,0.5692,2.6063,7.3152,...,-1.9889,0.0367,2.1225,3.7037,4.4815,-1.3830,0.7609,3.3797,3.6626,8.0384


OLS KITCHEN SINK  (5F)

In [45]:
params_1 = []
rsquared_1 = []
pvalues_1 = []
resid_1 = pd.DataFrame()
for i in port_excess_r.columns:
    res = sm.OLS(port_excess_r[i], Five_Factors_w_constant).fit()
    params_1.append(round(res.params,2))
    rsquared_1.append(round(res.rsquared,2))
    pvalues_1.append(round(res.pvalues,2))
    resid_1[f'res {i}'] = res.resid
    
rsquared_1 = pd.DataFrame(rsquared_1)
params_1 = pd.DataFrame(params_1)
pvalues_1 = pd.DataFrame(pvalues_1)

for i in Five_Factors_w_constant.columns:
    params_1[f'p-val {i}'] = pvalues_1[i]

params_1['R2'] = rsquared_1
params_1.index = portfolio.columns
params_1

,const,Mkt-RF,SMB,HML,RMW,CMA,p-val const,p-val Mkt-RF,p-val SMB,p-val HML,p-val RMW,p-val CMA,R2
SMALL LoBM,-0.26,1.05,1.26,-0.41,-0.50,-0.13,0.00,0.0,0.0,0.00,0.00,0.03,0.93
ME1 BM2,0.16,0.94,1.22,-0.18,-0.39,0.04,0.02,0.0,0.0,0.00,0.00,0.46,0.94
ME1 BM3,-0.00,0.93,1.07,0.13,-0.05,0.03,0.93,0.0,0.0,0.00,0.02,0.42,0.96
ME1 BM4,0.16,0.89,1.07,0.29,-0.04,0.06,0.00,0.0,0.0,0.00,0.11,0.11,0.95
SMALL HiBM,0.13,0.97,1.10,0.41,0.01,0.26,0.10,0.0,0.0,0.00,0.76,0.00,0.91
ME2 BM1,-0.08,1.10,0.98,-0.44,-0.17,-0.15,0.20,0.0,0.0,0.00,0.00,0.00,0.96
ME2 BM2,0.00,1.01,0.94,-0.03,0.08,0.02,0.92,0.0,0.0,0.17,0.00,0.52,0.95
ME2 BM3,-0.01,0.98,0.81,0.27,0.22,-0.00,0.88,0.0,0.0,0.00,0.00,0.99,0.94
ME2 BM4,0.02,0.96,0.75,0.42,0.09,0.07,0.73,0.0,0.0,0.00,0.00,0.03,0.95
ME2 BM5,0.00,1.08,0.89,0.62,0.04,0.04,0.98,0.0,0.0,0.00,0.11,0.26,0.95


REGRESSIONS: 3 FACTORS + MOMENTUM

In [46]:
Three_Factors_w_constant = sm.add_constant(Three_Factors)
Three_Factors_w_constant.insert(4, 'Mom', Mom)
Three_Factors_w_Mom = Three_Factors_w_constant
Three_Factors_w_Mom = Three_Factors_w_constant.iloc[:,:-1]

OLS KITCHEN SINK - 3f + MOM

In [47]:
params_2 = []
rsquared_2 = []
pvalues_2 = []
resid_2 = pd.DataFrame()
for i in port_excess_r.columns:
    res = sm.OLS(port_excess_r[i], Three_Factors_w_Mom).fit()
    params_2.append(round(res.params,2))
    rsquared_2.append(round(res.rsquared,2))
    pvalues_2.append(round(res.pvalues,2))
    resid_2[f'res {i}'] = res.resid
    
rsquared_2 = pd.DataFrame(rsquared_2)
params_2 = pd.DataFrame(params_2)
pvalues_2 = pd.DataFrame(pvalues_2)

for i in Three_Factors_w_Mom.columns:
    params_2[f'p-val {i}'] = pvalues_2[i]

params_2['R2'] = rsquared_2
params_2.index = portfolio.columns
params_2

,const,Mkt-RF,SMB,HML,Mom,p-val const,p-val Mkt-RF,p-val SMB,p-val HML,p-val Mom,R2
SMALL LoBM,-0.42,1.09,1.38,-0.30,-0.07,0.00,0.0,0.0,0.00,0.00,0.91
ME1 BM2,0.01,0.97,1.32,0.02,0.01,0.91,0.0,0.0,0.38,0.41,0.93
ME1 BM3,-0.01,0.93,1.09,0.30,-0.03,0.80,0.0,0.0,0.00,0.04,0.95
ME1 BM4,0.13,0.90,1.08,0.49,0.02,0.02,0.0,0.0,0.00,0.10,0.95
SMALL HiBM,0.17,0.95,1.11,0.69,-0.00,0.03,0.0,0.0,0.00,0.84,0.91
ME2 BM1,-0.13,1.13,1.01,-0.38,-0.06,0.04,0.0,0.0,0.00,0.00,0.95
ME2 BM2,0.05,1.01,0.90,0.11,-0.04,0.32,0.0,0.0,0.00,0.00,0.95
ME2 BM3,0.08,0.97,0.74,0.39,-0.03,0.17,0.0,0.0,0.00,0.02,0.93
ME2 BM4,0.05,0.95,0.73,0.57,0.00,0.35,0.0,0.0,0.00,0.78,0.95
ME2 BM5,0.01,1.09,0.88,0.78,0.00,0.90,0.0,0.0,0.00,0.89,0.95


REGRESSIONS: 3 FACTORS + ST REVERSAL

In [48]:
Three_Factors_w_constant = sm.add_constant(Three_Factors)
Three_Factors_w_constant.insert(4, 'ST REV', ST_Reversal)
Three_Factors_w_ST = Three_Factors_w_constant
Three_Factors_w_ST = Three_Factors_w_constant.iloc[:,:-1]

OLS FOR 3 FACTORS + ST REVERSAL

In [49]:
params_3 = []
rsquared_3 = []
pvalues_3 = []
resid_3 = pd.DataFrame()
for i in port_excess_r.columns:
    res = sm.OLS(port_excess_r[i], Three_Factors_w_ST).fit()
    params_3.append(round(res.params,2))
    rsquared_3.append(round(res.rsquared,2))
    pvalues_3.append(round(res.pvalues,2))
    resid_3[f'res {i}'] = res.resid
    
rsquared_3 = pd.DataFrame(rsquared_3)
params_3 = pd.DataFrame(params_3)
pvalues_3 = pd.DataFrame(pvalues_3)

for i in Three_Factors_w_ST.columns:
    params_3[f'p-val {i}'] = pvalues_3[i]

params_3['R2'] = rsquared_3
params_3.index = portfolio.columns
params_3

,const,Mkt-RF,SMB,HML,ST REV,p-val const,p-val Mkt-RF,p-val SMB,p-val HML,p-val ST REV,R2
SMALL LoBM,-0.49,1.10,1.38,-0.28,0.04,0.00,0.0,0.0,0.00,0.20,0.91
ME1 BM2,0.03,0.98,1.33,0.02,-0.04,0.66,0.0,0.0,0.38,0.14,0.93
ME1 BM3,-0.04,0.93,1.08,0.31,0.02,0.42,0.0,0.0,0.00,0.23,0.95
ME1 BM4,0.14,0.89,1.08,0.48,-0.00,0.00,0.0,0.0,0.00,0.82,0.95
SMALL HiBM,0.15,0.94,1.10,0.69,0.05,0.04,0.0,0.0,0.00,0.06,0.91
ME2 BM1,-0.19,1.13,1.00,-0.36,0.03,0.00,0.0,0.0,0.00,0.22,0.95
ME2 BM2,0.03,1.02,0.91,0.13,-0.03,0.59,0.0,0.0,0.00,0.12,0.95
ME2 BM3,0.04,0.97,0.74,0.40,0.03,0.46,0.0,0.0,0.00,0.08,0.93
ME2 BM4,0.05,0.96,0.73,0.57,-0.01,0.28,0.0,0.0,0.00,0.51,0.95
ME2 BM5,0.01,1.08,0.88,0.78,0.01,0.92,0.0,0.0,0.00,0.52,0.95


In [50]:
np.mean(params_3['const'])*12

-0.21599999999999994

GRS

In [51]:
mean_F1 = Five_Factors_w_constant["Mkt-RF"] - Five_Factors_w_constant["Mkt-RF"].mean(axis=0)
mean_F2 = Five_Factors_w_constant["SMB"] - Five_Factors_w_constant["SMB"].mean(axis=0)
mean_F3 = Five_Factors_w_constant["HML"] - Five_Factors_w_constant["HML"].mean(axis=0)
mean_F4 = Five_Factors_w_constant["RMW"] - Five_Factors_w_constant["RMW"].mean(axis=0)
mean_F5 = Five_Factors_w_constant["CMA"] - Five_Factors_w_constant["CMA"].mean(axis=0)
matrix = np.zeros((5,5))
for i in range(len(Five_Factors_w_constant)):
    vector = np.array([mean_F1[i], mean_F2[i], mean_F3[i], mean_F4[i], mean_F5[i]])
    matrix = matrix + np.outer(vector, vector)
omega_inv = np.linalg.inv(matrix/len(Five_Factors_w_constant))
factor_means = np.array([Five_Factors_w_constant["Mkt-RF"].mean(axis=0), Five_Factors_w_constant["SMB"].mean(axis=0), Five_Factors_w_constant["HML"].mean(axis=0), Five_Factors_w_constant["RMW"].mean(axis=0), Five_Factors_w_constant["CMA"].mean(axis=0)])
t = len(portfolio)
n = len(portfolio.columns)
k = (len(Five_Factors_w_constant.columns)-1)
alpha = params_1.iloc[:,0]

covariance_resid = np.linalg.inv((resid_1.T @ resid_1)/t)
GRS_stat_5F = ((t - n - k)/n)*(1/(1 + factor_means.T @ omega_inv @ factor_means))*(alpha.T @ covariance_resid @ alpha)
p_value_5F = 1 - sp.stats.f.cdf(GRS_stat_5F, n, (t-n-k))
print(f"GRS stat 5 Factors: {GRS_stat_5F} ; p-value: {p_value_5F}")

GRS stat 5 Factors: 2.8898465800490754 ; p-value: 4.176751097761944e-06


In [69]:
mean_F1 = Three_Factors_w_Mom["Mkt-RF"] - Three_Factors_w_Mom["Mkt-RF"].mean(axis=0)
mean_F2 = Three_Factors_w_Mom["SMB"] - Three_Factors_w_Mom["SMB"].mean(axis=0)
mean_F3 = Three_Factors_w_Mom["HML"] - Three_Factors_w_Mom["HML"].mean(axis=0)
mean_F4 = Three_Factors_w_Mom["Mom"] - Three_Factors_w_Mom["Mom"].mean(axis=0)
matrix = np.zeros((4,4))
for i in range(len(Three_Factors_w_Mom)):
    vector = np.array([mean_F1[i], mean_F2[i], mean_F3[i], mean_F4[i]])
    matrix = matrix + np.outer(vector, vector)
omega_inv = np.linalg.inv(matrix/len(Three_Factors_w_Mom))
factor_means = np.array([Three_Factors_w_Mom["Mkt-RF"].mean(axis=0), Three_Factors_w_Mom["SMB"].mean(axis=0), Three_Factors_w_Mom["HML"].mean(axis=0), Three_Factors_w_Mom["Mom"].mean(axis=0)])
t = len(portfolio)
n = len(portfolio.columns)
k = (len(Three_Factors_w_Mom.columns)-1)
alpha = params_2.iloc[:,0]

covariance_resid = np.linalg.inv((resid_2.T @ resid_2)/t)
GRS_stat_3F_Mom = ((t - n - k)/n)*(1/(1 + factor_means.T @ omega_inv @ factor_means))*(alpha.T @ covariance_resid @ alpha)
p_value_3F_Mom = 1 - sp.stats.f.cdf(GRS_stat_3F_Mom, n, (t-n-k))
print(f"GRS stat 3 Factors + Mom: {GRS_stat_3F_Mom} ; p-value: {p_value_3F_Mom}")


GRS stat 3 Factors + Mom: 24.729243024872847 ; p-value: 1.1102230246251565e-16


In [54]:
from numpy.linalg import inv

mean_F1 = Three_Factors_w_ST["Mkt-RF"] - Three_Factors_w_ST["Mkt-RF"].mean(axis=0)
mean_F2 = Three_Factors_w_ST["SMB"] - Three_Factors_w_ST["SMB"].mean(axis=0)
mean_F3 = Three_Factors_w_ST["HML"] - Three_Factors_w_ST["HML"].mean(axis=0)
mean_F4 = Three_Factors_w_ST["ST REV"] - Three_Factors_w_ST["ST REV"].mean(axis=0)
matrix = np.zeros((4,4))
for i in range(len(Three_Factors_w_ST)):
    vector = np.array([mean_F1[i], mean_F2[i], mean_F3[i], mean_F4[i]])
    matrix = matrix + np.outer(vector, vector)
omega_inv = np.linalg.inv(matrix/len(Three_Factors_w_ST))
factor_means = np.array([Three_Factors_w_ST["Mkt-RF"].mean(axis=0), Three_Factors_w_ST["SMB"].mean(axis=0), Three_Factors_w_ST["HML"].mean(axis=0), Three_Factors_w_ST["ST REV"].mean(axis=0)])
t = len(portfolio)
n = len(portfolio.columns)
k = (len(Three_Factors_w_ST.columns)-1)
alpha = params_3.iloc[:,0]

covariance_resid = np.linalg.inv((resid_3.T @ resid_3)/t)
GRS_stat_3F_ST = ((t - n - k)/n)*(1/(1 + factor_means.T @ omega_inv @ factor_means))*(alpha.T @ covariance_resid @ alpha)
p_value_3F_ST = 1 - sp.stats.f.cdf(GRS_stat_3F_ST, n, (t-n-k))
print(f"GRS stat 3 Factors + ST: {GRS_stat_3F_ST} ; p-value: {p_value_3F_ST}")

GRS stat 3 Factors + ST: 3.7751407157760384 ; p-value: 3.1176996539983293e-09


In [55]:
x = [GRS_stat_5F, p_value_5F]
y = [GRS_stat_3F_Mom, p_value_3F_Mom]
z = [GRS_stat_3F_ST, p_value_3F_ST]
w = [x,y,z]
w = pd.DataFrame(w)
w.columns = ['GRS','p-Value']
w.index = ['5F','3F + Mom', '3F + ST']
w

,GRS,p-Value
5F,2.889847,4.176751e-06
3F + Mom,2.968347,2.249892e-06
3F + ST,3.775141,3.117700e-09


FAMA MACBETH CROSS SECTIONAL REGRESSION

In [56]:
params_1 = sm.add_constant(params_1)
params_2 = sm.add_constant(params_2)
params_3 = sm.add_constant(params_3)
params_1.iloc[:,1] = params_1.iloc[:,0]
params_2.iloc[:,1] = params_2.iloc[:,0]
params_3.iloc[:,1] = params_3.iloc[:,0]
params_cross_sectional = [params_1.iloc[:,1:7], params_2.iloc[:,1:6], params_3.iloc[:,1:6]]

In [57]:
x = port_excess_r.T

Five_Factors_ff = []
Three_Mom_ff = []
Three_ST_ff = []
coeff_ff = [Five_Factors_ff,Three_Mom_ff,Three_ST_ff]
for i in range(3):
    for column in x.columns:
        y = sm.OLS(x[column],params_cross_sectional[i]).fit()
        coeff_ff[i].append(y.params)


In [58]:
Five_Factors_ff = pd.DataFrame(Five_Factors_ff)
Three_Mom_ff = pd.DataFrame(Three_Mom_ff)
Three_ST_ff = pd.DataFrame(Three_ST_ff)
np.sum(Three_ST_ff['const'])

761.0906230504913

FIND RISK PREMIA AND PRICING ERROR

In [59]:
coeff = [Five_Factors_ff, Three_Mom_ff, Three_ST_ff]
lamda_five_ff = []
lamda_three_mom = []
lamda_three_st = []
lamdas = [lamda_five_ff, lamda_three_mom, lamda_three_st]
for i in range(3):
    for column in coeff[i].columns:
        lamdas_j = np.mean(coeff[i][column])
        lamdas[i].append(lamdas_j)

In [60]:
lamda_five_ff = pd.DataFrame(lamda_five_ff)
lamda_three_mom = pd.DataFrame(lamda_three_mom)
lamda_three_st = pd.DataFrame(lamda_three_st)

In [61]:
lamda_five_ff.index = params_1.iloc[:,1:7].columns
lamda_five_ff.columns = ['Five Factors']

lamda_three_mom.index = params_2.iloc[:,1:6].columns
lamda_three_mom.columns = ['Three + Mom']

lamda_three_st.index = params_3.iloc[:,1:6].columns
lamda_three_st.columns = ['Three + ST']

In [62]:
lamda_five_ff#.iloc[2]

,Five Factors
const,0.801848
Mkt-RF,-0.236530
SMB,0.288837
HML,0.248173
RMW,0.356577
CMA,0.131942


COMPUTE T STATISTIC

In [63]:
len(Five_Factors_ff)

696

In [64]:
t_stat_five_ff = []
for i in range(len(lamda_five_ff)):
    t_stat_i = lamda_five_ff.iloc[i]/(np.std(Five_Factors_ff.iloc[:,i])/np.sqrt(len(Five_Factors_ff))) 
    t_stat_five_ff.append(t_stat_i)

t_stat_three_mom = []
for i in range(len(lamda_three_mom)):
    t_stat_i = lamda_three_mom.iloc[i]/(np.std(Three_Mom_ff.iloc[:,i]/np.sqrt(len(Three_Mom_ff))))
    t_stat_three_mom.append(t_stat_i)

t_stat_three_st = []
for i in range(len(lamda_three_st)):
    t_stat_i = lamda_three_st.iloc[i]/(np.std(Three_ST_ff.iloc[:,i])/np.sqrt(len(Three_ST_ff)))
    t_stat_three_st.append(t_stat_i)

In [65]:
t_stat_five_ff = pd.DataFrame(t_stat_five_ff)
t_stat_three_mom = pd.DataFrame(t_stat_three_mom)
t_stat_three_st = pd.DataFrame(t_stat_three_st)

In [66]:
five_factors_final_table= pd.concat([lamda_five_ff, t_stat_five_ff], axis = 1)
five_factors_final_table.columns = ['lamda', 't-stat']

three_factor_and_mom_final_table= pd.concat([lamda_three_mom, t_stat_three_mom], axis = 1)
three_factor_and_mom_final_table.columns = ['lamda', 't-stat']

three_factor_and_ST_final_table= pd.concat([lamda_three_st, t_stat_three_st], axis = 1)
three_factor_and_ST_final_table.columns = ['lamda', 't-stat']

three_factor_and_ST_final_table 
five_factors_final_table 

,lamda,t-stat
const,0.801848,2.970661
Mkt-RF,-0.236530,-0.740976
SMB,0.288837,2.455903
HML,0.248173,2.200952
RMW,0.356577,2.100055
CMA,0.131942,0.821861


In [67]:
three_factor_and_ST_final_table

,lamda,t-stat
const,1.093521,4.131814
Mkt-RF,-0.474521,-1.512999
SMB,0.174186,1.466443
HML,0.307032,2.731500
ST REV,-0.946680,-1.997044


In [68]:
three_factor_and_mom_final_table

,lamda,t-stat
const,0.200456,0.667337
Mkt-RF,0.457491,1.311542
SMB,0.174967,1.472242
HML,0.331219,2.941091
Mom,2.883297,3.978572
